<img src="../img/Logo.png" width="200">

# Aprendizaje por Imágenes
## Optimizadores

### Profesor: Jorge Calvo

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt

In [ ]:
# Cargar el dataset MNIST y preprocesar las imágenes
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Reestructuramos y normalizamos (las imágenes se escalan a [0, 1])
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# Visualizar algunas imágenes de entrenamiento
fig, axs = plt.subplots(2, 4, figsize=(8, 4))
for i, ax in enumerate(axs.flat):
    ax.imshow(x_train[i])
    ax.set_title(f"Label: {y_train[i]}")
    ax.axis('off')

plt.tight_layout()
plt.show()

### Explicación de Optimizadores

#### AdaGrad:
* Ventaja: Adapta bien la tasa de aprendizaje para parámetros con gradientes escasos (útil cuando algunas características visuales aparecen con menor frecuencia).
* Desventaja: La tasa de aprendizaje puede disminuir demasiado con el tiempo.

#### RMSProp:

* Ventaja: Mantiene una tasa de aprendizaje estable mediante un promedio móvil de los gradientes, lo que es beneficioso en escenarios con funciones de pérdida muy variables, como es frecuente en redes convolucionales.
* Desventaja: Requiere ajustar el hiperparámetro de decaimiento (𝛾).

#### Adam:

* Ventaja: Combina la estabilidad de RMSProp y la eficiencia del momentum, lo que resulta en una convergencia rápida y robusta en una amplia variedad de problemas de visión artificial.
* Desventaja: Aunque suele funcionar bien en la mayoría de los casos, en algunos escenarios específicos puede requerir ajustes finos de la tasa de aprendizaje.

In [ ]:
# Función para construir la arquitectura del modelo
def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

# Diccionario con los optimizadores a comparar
optimizers_dict = {
    "Adam": optimizers.Adam(learning_rate=0.001),
    "AdaGrad": optimizers.Adagrad(learning_rate=0.001),
    "RMSProp": optimizers.RMSprop(learning_rate=0.001)
}

epochs = 10
batch_size = 128

# Fórmulas de Cross Entropy y Sparse Categorical Cross Entropy

### Categorical Cross Entropy

La función de pérdida de **Categorical Cross Entropy** se define como:

$$
L = -\sum_{i=1}^{C} y_i \log(\hat{y}_i)
$$

donde:
- \( C \) es el número
 clases.
- \( y_i \) es la etiqueta verdadera para la clase \( i \) (en formato one-hot, es decir, \( y_i = 1 \) para la clase correcta y \( 0 \) para
s demás).
- \( \hat{y}_i \) es la probabilidad predicha para la c

#### Ventajas:

Directamente optimizada para clasificación.
Penaliza fuertemente las predicciones incorrectas.
La salida softmax genera una distribución de probabilidad que se alinea con la función de pérdida.

* Aplicación en MNIST:
Es la función de pérdida más utilizada en tareas de clasificación multiclase, ya que proporciona un gradiente bien definido para la actualización de los pesos.lase \( i \).

### Sparse Categorical Cross Entropy

La función de pérdida de **Sparse Categorical Cross Entropy** se define como:

$$
L = -\log\left(\hat{y}_{y_{\text{true}}}\right)
$$

donde:
- \( \hat{y}_{y_{\text{true}}} \) es la probabilidad predicha para la clase correcta.
- Las etiquetas verdaderas se representan como enteros (por ejemplo, \( 3 \) en lugar de \(

#### Ventajas:

Menos consumo de memoria y computacionalmente más eficiente cuando se tienen muchas clases.
Evita el paso de transformación a one-hot, lo cual simplifica el preprocesamiento.

* Aplicación en MNIST:
Debido a que MNIST tiene 10 clases y las etiquetas pueden representarse fácilmente como enteros, es común usar sparse categorical cross entropy para simplificar el código y la preparación de datos.[0,0,0,1,0,0,0,0,0,0]\)).


In [ ]:
# Almacenaremos los historiales de entrenamiento para cada optimizador
histories = {}

for opt_name, opt in optimizers_dict.items():
    print(f"Entrenando con el optimizador: {opt_name}")
    model = build_model()
    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',  # Utilizamos sparse ya que las etiquetas son enteros
                  metrics=['accuracy'])
    history = model.fit(x_train, y_train,
                        validation_data=(x_test, y_test),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=1)
    histories[opt_name] = history

In [ ]:
# Visualización de las curvas de entrenamiento y validación para precisión y pérdida
plt.figure(figsize=(14, 6))

# Curva de precisión
plt.subplot(1, 2, 1)
for opt_name, history in histories.items():
    plt.plot(history.history['accuracy'], label=f'{opt_name} - Entrenamiento')
    plt.plot(history.history['val_accuracy'], label=f'{opt_name} - Validación', linestyle='--')
plt.title('Evolución de la Precisión')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()

# Curva de pérdida
plt.subplot(1, 2, 2)
for opt_name, history in histories.items():
    plt.plot(history.history['loss'], label=f'{opt_name} - Entrenamiento')
    plt.plot(history.history['val_loss'], label=f'{opt_name} - Validación', linestyle='--')
plt.title('Evolución de la Pérdida')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()

plt.tight_layout()
plt.show()